In [2]:
import chess
import torch
import numpy as np

# Mapping chess pieces to one-hot encoded vectors (12 possible values)
piece_to_one_hot = {
    'P': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # White Pawn
    'N': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # White Knight
    'B': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # White Bishop
    'R': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],  # White Rook
    'Q': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],  # White Queen
    'K': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],  # White King
    'p': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],  # Black Pawn
    'n': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],  # Black Knight
    'b': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],  # Black Bishop
    'r': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],  # Black Rook
    'q': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],  # Black Queen
    'k': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],  # Black King
    '.': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]   # Empty square
}

def board_to_tensor(board):
    # Create an 8x8 matrix to represent the board, where each square gets a one-hot encoded vector
    board_matrix = []
    for rank in board.board_fen().split('/'):
        row = []
        for char in rank:
            if char.isdigit():
                row.extend([piece_to_one_hot['.']] * int(char))  # Empty squares
            else:
                row.append(piece_to_one_hot[char])  # Piece type
        board_matrix.append(row)
    
    # Convert the list of lists into a tensor
    tensor = torch.tensor(board_matrix, dtype=torch.float32)
    return tensor

# Example usage
board = chess.Board()
tensor = board_to_tensor(board)
print(tensor.shape)  # Output tensor shape


torch.Size([8, 8, 12])


In [10]:
import chess
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

def extract_material_vector(board):
    # Initialize counts for the 14 piece types
    counts = [0] * 14  # [P, N, R, Q, K, LB, DB, p, n, r, q, k, lb, db]

    # Loop through all squares on the board
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if not piece:
            continue  # Skip empty squares

        color = piece.color  # True = white, False = black
        is_light_square = (square % 2 == 0) ^ (square // 8 % 2 == 0)

        # Handling bishops differently based on color of square
        if piece.piece_type == chess.BISHOP:
            if color:  # White
                if is_light_square:
                    counts[5] += 1  # Light-squared bishop (LB)
                else:
                    counts[6] += 1  # Dark-squared bishop (DB)
            else:  # Black
                if is_light_square:
                    counts[12] += 1  # Light-squared bishop (lb)
                else:
                    counts[13] += 1  # Dark-squared bishop (db)

        elif piece.piece_type == chess.PAWN:
            counts[0 if color else 7] += 1  # White pawns (index 0) or black pawns (index 7)
        elif piece.piece_type == chess.KNIGHT:
            counts[1 if color else 8] += 1  # White knights (index 1) or black knights (index 8)
        elif piece.piece_type == chess.ROOK:
            counts[2 if color else 9] += 1  # White rooks (index 2) or black rooks (index 9)
        elif piece.piece_type == chess.QUEEN:
            counts[3 if color else 10] += 1  # White queens (index 3) or black queens (index 10)
        elif piece.piece_type == chess.KING:
            counts[4 if color else 11] += 1  # White kings (index 4) or black kings (index 11)

    # Convert the list of counts to a PyTorch tensor
    return torch.tensor(counts, dtype=torch.float32)

# Neural Network Model
class MaterialNet(nn.Module):
    def __init__(self):
        super(MaterialNet, self).__init__()
        
        # Shared layer for both white and black pieces
        self.shared_layer = nn.Linear(7, 16)  # 7 input features for each side, 16 output features
        
        # Mixed layer after concatenating white and black piece outputs
        self.mixed_layer = nn.Linear(32, 16)  # 32 input features (16 from white + 16 from black)
        
        # Output layer
        self.fc_out = nn.Linear(16, 1)  # Final output: a scalar (e.g., centipawn evaluation)

    def forward(self, x):
        # Split input material vector for white and black pieces
        white_counts = x[:7]  # First 7 elements for white pieces
        black_counts = x[7:]  # Next 7 elements for black pieces
        
        # Process both white and black piece counts through the shared layer independently
        white_output = F.relu(self.shared_layer(white_counts))
        black_output = F.relu(self.shared_layer(black_counts))
        
        # Concatenate the outputs from both sides (white and black pieces)
        combined_output = torch.cat((white_output, black_output), dim=0)
        
        # Pass combined output through the mixed layer
        mixed_output = F.relu(self.mixed_layer(combined_output))
        
        # Final output: scalar evaluation (centipawn or similar)
        evaluation = self.fc_out(mixed_output)
        
        return evaluation

# Example usage
board = chess.Board()
material_vector = extract_material_vector(board)
print(material_vector)

# Create an instance of the network and run the material vector through it
model = MaterialNet()
output = model(material_vector)
print(output)


tensor([8., 2., 2., 1., 1., 1., 1., 8., 2., 2., 1., 1., 1., 1.])
tensor([-0.3480], grad_fn=<ViewBackward0>)


In [ ]:
# Pawn hash table - pawn and king position as a means of evaluating structure.
# Piece square table - assigns values to pieces on different squares
# 

# Lazy evaluation where above or below certain threshold, the value is just returned. 
# A network that decides when to use which paramaters. 